# Import Library & Load Data

In [11]:
import pandas as pd

from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.proxy import Proxy, ProxyType
import random
from plyer import notification

In [2]:
company_alias_null = pd.read_csv('../data/generated_data/company_alias.csv')
company_alias_null = company_alias_null.rename(columns={'company_name': 'c1', 'alias_company_name': 'c2'})
company_alias_null["c1_context"] = None
company_alias_null["c2_context"] = None
company_alias_null = company_alias_null[["c1", "c1_context", "c2", "c2_context", "match_type"]]
company_alias_null = company_alias_null.drop_duplicates(subset=['c1', 'c2'])
company_alias_null

,c1,c1_context,c2,c2_context,match_type
0,anz,None,ANZ,None,Match
1,anz,None,Australian New Zealand Banking Group,None,Match
2,anz,None,A.N.Z.,None,Match
3,anz,None,ANZ Bank,None,Match
4,anz,None,Anz,None,Match
...,...,...,...,...,...
32864,michigan state university,None,University of Michigan,None,Non-Match
32865,michigan state university,None,Michigan Technological University,None,Non-Match
32866,michigan state university,None,Michigan State College,None,Non-Match
32867,michigan state university,None,Central Michigan University,None,Non-Match


In [3]:
company_alias_null.to_csv('../data/data_context/data_with_context.csv', index=False)

In [4]:
# Merge company alias and data with context
data_with_context = pd.read_csv('../data/data_context/data_with_context.csv')

company_alias = company_alias_null.merge(data_with_context[['c1', 'c1_context', 'c2', 'c2_context']], on=['c1', 'c2'], how='left')
company_alias['c1_context'] = company_alias['c1_context_x'].combine_first(company_alias['c1_context_y'])
company_alias['c2_context'] = company_alias['c2_context_x'].combine_first(company_alias['c2_context_y'])
company_alias = company_alias[['c1', 'c1_context', 'c2', 'c2_context', 'match_type']]
company_alias

C:\Users\mfauz\AppData\Local\Temp\ipykernel_31848\2552144561.py:5: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  company_alias['c1_context'] = company_alias['c1_context_x'].combine_first(company_alias['c1_context_y'])
C:\Users\mfauz\AppData\Local\Temp\ipykernel_31848\2552144561.py:6: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  company_alias['c2_context'] = company_alias['c2_context_x'].combine_first(company_alias['c2_context_y'])


,c1,c1_context,c2,c2_context,match_type
0,anz,NaN,ANZ,NaN,Match
1,anz,NaN,Australian New Zealand Banking Group,NaN,Match
2,anz,NaN,A.N.Z.,NaN,Match
3,anz,NaN,ANZ Bank,NaN,Match
4,anz,NaN,Anz,NaN,Match
...,...,...,...,...,...
32556,michigan state university,NaN,University of Michigan,NaN,Non-Match
32557,michigan state university,NaN,Michigan Technological University,NaN,Non-Match
32558,michigan state university,NaN,Michigan State College,NaN,Non-Match
32559,michigan state university,NaN,Central Michigan University,NaN,Non-Match


In [12]:
company_alias = pd.read_csv('../data/data_context/data_with_context.csv')
# company_alias = pd.read_csv('../data/data_context/data_done.csv')
company_alias['c1_context'] = company_alias['c1_context'].replace('No Desc', None)
company_alias['c2_context'] = company_alias['c2_context'].replace('No Desc', None)
company_alias

# # F salah satu dari c1_context dan c2_context null, maka replace null tersebut jadi No Desc, Kalo keduanya null, maka biarkan
# company_alias['c1_context'] = company_alias.apply(lambda row: 'No Desc' if pd.isnull(row['c1_context']) and not pd.isnull(row['c2_context']) else row['c1_context'], axis=1)
# company_alias['c2_context'] = company_alias.apply(lambda row: 'No Desc' if pd.isnull(row['c2_context']) and not pd.isnull(row['c1_context']) else row['c2_context'], axis=1)

,c1,c1_context,c2,c2_context,match_type
0,aegon nederland,NaN,Aegon,NaN,Match
1,aegon nederland,NaN,Aegon NV,NaN,Match
2,aegon nederland,NaN,Aegon NLD,NaN,Match
3,aegon nederland,NaN,Aegon Nederland BV,NaN,Match
4,aegon nederland,NaN,Aegaon Nederland,NaN,Match
...,...,...,...,...,...
5834,michigan state university,NaN,MSU,NaN,Match
5835,michigan state university,NaN,Mich. State Univ.,NaN,Match
5836,michigan state university,NaN,Michigan St. University,NaN,Match
5837,michigan state university,NaN,MichiganStateUniversity,NaN,Match


In [13]:
def send_windows_notification():
    notification.notify(
        title='CAPTCHA Detected',
        message='Please solve the CAPTCHA manually.',
        app_name='CAPTCHA Solver',
        timeout=10  # Notification will disappear after 10 seconds
    )

In [17]:
def setup_webdriver():
    options = webdriver.ChromeOptions()
    # options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-extensions")
    options.add_argument("--proxy-server='direct://'")
    options.add_argument("--proxy-bypass-list=*")
    options.add_argument("--start-maximized")
    options.add_argument("--disable-infobars")
    options.add_argument("--disable-browser-side-navigation")
    options.add_argument("--disable-extensions")
    options.add_argument("--disable-notifications")
    options.add_argument("--incognito")
    options.add_argument("--disable-webrtc")
    options.add_argument('--disable-blink-features=AutomationControlled')
    driver = webdriver.Chrome(service=Service(
        ChromeDriverManager().install()), options=options)
    return driver

def check_and_retry(driver):
    try:
        # Cari elemen yang menunjukkan bahwa halaman CAPTCHA muncul
        element = driver.find_element(By.XPATH, "//b[contains(text(), 'About this page')]")
        if element:
            print("Encountered 'About this page.' CAPTCHA detected. Please solve the CAPTCHA manually.")
            send_windows_notification()
            
            # Tunggu sampai CAPTCHA selesai
            while True:
                input("Press Enter after you have solved the CAPTCHA and the page has loaded...")
                try:
                    # Coba verifikasi apakah halaman setelah CAPTCHA berhasil dimuat
                    driver.find_element(By.CSS_SELECTOR, "#APjFqb")
                    print("CAPTCHA solved. Resuming scraping...")
                    break  # Keluar dari loop saat CAPTCHA berhasil diselesaikan
                except NoSuchElementException:
                    print("CAPTCHA might not be solved yet. Please solve it and press Enter.")
                    sleep(3)  # Tunggu 3 detik sebelum memeriksa lagi
                    continue  # Jika elemen belum ditemukan, lanjutkan menunggu

            return False  # Tidak perlu mengulang setup webdriver, cukup lanjutkan proses
    except NoSuchElementException:
        print("No CAPTCHA detected.")
        return False  # Tidak perlu mengulang setup webdriver, cukup lanjutkan proses

    return False

def getContext(driver, company_name):
    try:
        search_bar = driver.find_element(By.CSS_SELECTOR, "#APjFqb")
        search_bar.clear()
        search_bar.send_keys(f'"{company_name}" company description')
        # search_bar.send_keys(f"'{company_name}' company description")
        search_bar.send_keys(Keys.ENTER)
        
        # sleep(random.uniform(5, 7))
        sleep(1)
        
        # Get Context dari summary
        try:
            context_element = driver.find_element(By.XPATH, "//div[@data-md='61']")
        except NoSuchElementException:
            context_element = None
        
        if context_element and context_element.text:
            return context_element.text
        else:
            try:
                # Kalau gaada summary, ambil dari desc site teratas
                context_element = driver.find_element(By.CSS_SELECTOR, "#rso > div:nth-child(1) > div > div > div > div:nth-child(2) > div > span")
            except NoSuchElementException:
                context_element = None
            
            if context_element and context_element.text:
                return context_element.text
            else:
                return "No Desc"

    except NoSuchElementException:
        return "No Desc"

# Loop untuk memproses setiap baris dataset
while True:
    driver = setup_webdriver()

    # Langkah-langkah inisialisasi (setting bahasa, region, dan search pertama)
    driver.get("https://www.google.com/preferences?prev=https://www.google.com/search?q%3Dtes%26sca_esv%3D36f6d775f0bd8871%26sca_upv%3D1%26source%3Dhp%26ei%3DZejSZonQCIOC4-EP05nBoAI%26iflsig%3DAL9hbdgAAAAAZtL2dXrPEGrsWqkKK3N2rPzD7oVCPYzO%26ved%3D0ahUKEwjJlsWU-56IAxUDwTgGHdNMECQQ4dUDCA4%26uact%3D5%26oq%3Dtes%26gs_lp%3DEgdnd3Mtd2l6IgN0ZXMyEBAAGIAEGLEDGEMYgwEYigUyCBAAGIAEGLEDMggQABiABBixAzIIEAAYgAQYsQMyCBAAGIAEGLEDMgsQABiABBixAxiDATIQEAAYgAQYsQMYQxiDARiKBTIFEAAYgAQyBRAAGIAEMgUQABiABEjfAlAAWP8BcAB4AJABAJgBPqABmQGqAQEzuAEDyAEA-AEBmAIDoAKjAcICChAAGIAEGEMYigXCAhEQLhiABBixAxjRAxiDARjHAcICEBAuGIAEGNEDGEMYxwEYigWYAwCSBwEzoAe7FA%26sclient%3Dgws-wiz&lang=1&prev=https://www.google.com/preferences?hl%3Did%26prev%3Dhttps://www.google.com/search%253Fq%253Dtes%2526sca_esv%253D36f6d775f0bd8871%2526sca_upv%253D1%2526source%253Dhp%2526ei%253DZejSZonQCIOC4-EP05nBoAI%2526iflsig%253DAL9hbdgAAAAAZtL2dXrPEGrsWqkKK3N2rPzD7oVCPYzO%2526ved%253D0ahUKEwjJlsWU-56IAxUDwTgGHdNMECQQ4dUDCA4%2526uact%253D5%2526oq%253Dtes%2526gs_lp%253DEgdnd3Mtd2l6IgN0ZXMyEBAAGIAEGLEDGEMYgwEYigUyCBAAGIAEGLEDMggQABiABBixAzIIEAAYgAQYsQMyCBAAGIAEGLEDMgsQABiABBixAxiDATIQEAAYgAQYsQMYQxiDARiKBTIFEAAYgAQyBRAAGIAEMgUQABiABEjfAlAAWP8BcAB4AJABAJgBPqABmQGqAQEzuAEDyAEA-AEBmAIDoAKjAcICChAAGIAEGEMYigXCAhEQLhiABBixAxjRAxiDARjHAcICEBAuGIAEGNEDGEMYxwEYigWYAwCSBwEzoAe7FA%2526sclient%253Dgws-wiz")
    sleep(2)

    # Setting up language and region
    driver.find_element(By.CSS_SELECTOR, "body > div.iORcjf > div:nth-child(2) > div > div:nth-child(2) > div > div:nth-child(2) > div > div > div.HrqWPb").click()
    driver.find_element(By.CSS_SELECTOR, "#lb > div > div.mcPPZ.nP0TDe.xg7rAe.ivkdbf > span > div > g-text-field > div.WO1lOd > div.FFTibe > input").send_keys("English")
    driver.find_element(By.CSS_SELECTOR, "#lb > div > div.mcPPZ.nP0TDe.xg7rAe.ivkdbf > span > div > g-menu > g-menu-item:nth-child(2)").click()
    driver.find_element(By.CSS_SELECTOR, "#lb > div > div.mcPPZ.nP0TDe.xg7rAe.ivkdbf > span > div > div.JhVSze > span:nth-child(2)").click()

    sleep(2)

    driver.find_element(By.CSS_SELECTOR, "body > div.iORcjf > div:nth-child(2) > div > div:nth-child(2) > div > div:nth-child(2) > div > div:nth-child(2) > div.HrqWPb > div").click()
    driver.find_element(By.CSS_SELECTOR, "#lb > div > div.mcPPZ.nP0TDe.xg7rAe.ivkdbf > span > div > g-text-field > div.WO1lOd > div.FFTibe > input").send_keys("United States")
    driver.find_element(By.CSS_SELECTOR, "#lb > div > div.mcPPZ.nP0TDe.xg7rAe.ivkdbf > span > div > g-menu > g-menu-item:nth-child(179) > div").click()
    driver.find_element(By.CSS_SELECTOR, "#lb > div > div.mcPPZ.nP0TDe.xg7rAe.ivkdbf > span > div > div.JhVSze > span:nth-child(2)").click()

    driver.get("https://google.com")
    sleep(2)
    initial_search = driver.find_element(By.CSS_SELECTOR, "#APjFqb")
    initial_search.send_keys("company")
    initial_search.send_keys(Keys.ENTER)

    # Check if we need to restart the driver
    if check_and_retry(driver):
        continue

    # Proses dataset
    for index, row in company_alias.iterrows():

        # Check Captcha
        if check_and_retry(driver): 
            continue

        c1_value = row['c1']
        if pd.isnull(row['c1_context']):
            previous_rows = company_alias.loc[:index-1]
            if not previous_rows[previous_rows['c1'] == c1_value].empty:
                previous_c1_context = previous_rows[previous_rows['c1'] == c1_value]['c1_context'].values[0]
                if pd.notnull(previous_c1_context):
                    company_alias.at[index, 'c1_context'] = previous_c1_context
                else:
                    company_alias.at[index, 'c1_context'] = getContext(driver, c1_value)
            else:
                company_alias.at[index, 'c1_context'] = getContext(driver, c1_value)

        c2_value = row['c2']
        if pd.isnull(row['c2_context']):
            previous_rows = company_alias.loc[:index-1]
            if not previous_rows[previous_rows['c2'] == c2_value].empty:
                previous_c2_context = previous_rows[previous_rows['c2'] == c2_value]['c2_context'].values[0]
                if pd.notnull(previous_c2_context):
                    company_alias.at[index, 'c2_context'] = previous_c2_context
                else:
                    company_alias.at[index, 'c2_context'] = getContext(driver, c2_value)
            else:
                company_alias.at[index, 'c2_context'] = getContext(driver, c2_value)

        # company_alias.to_csv('../data/data_context/data_with_context.csv', index=False)
        company_alias.to_csv('../data/data_context/data_done.csv', index=False)
        print("Data ke- " + str(index))


    break


No CAPTCHA detected.
No CAPTCHA detected.
Data ke- 0
No CAPTCHA detected.
Data ke- 1
No CAPTCHA detected.
Data ke- 2
No CAPTCHA detected.
Data ke- 3
No CAPTCHA detected.
Data ke- 4
No CAPTCHA detected.
Data ke- 5
No CAPTCHA detected.
Data ke- 6
No CAPTCHA detected.
Data ke- 7
No CAPTCHA detected.
Data ke- 8
No CAPTCHA detected.
Data ke- 9
No CAPTCHA detected.
Data ke- 10
No CAPTCHA detected.
Data ke- 11
No CAPTCHA detected.
Data ke- 12
No CAPTCHA detected.
Data ke- 13
No CAPTCHA detected.
Data ke- 14
No CAPTCHA detected.
Data ke- 15
No CAPTCHA detected.
Data ke- 16
No CAPTCHA detected.
Data ke- 17
No CAPTCHA detected.
Data ke- 18
No CAPTCHA detected.
Data ke- 19
No CAPTCHA detected.
Data ke- 20
No CAPTCHA detected.
Data ke- 21
No CAPTCHA detected.
Data ke- 22
No CAPTCHA detected.
Data ke- 23
No CAPTCHA detected.
Data ke- 24
No CAPTCHA detected.
Data ke- 25
No CAPTCHA detected.
Data ke- 26
No CAPTCHA detected.
Data ke- 27
No CAPTCHA detected.
Data ke- 28
No CAPTCHA detected.
Data ke- 29

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=128.0.6613.121)
Stacktrace:
	GetHandleVerifier [0x008CD933+25811]
	(No symbol) [0x0085E314]
	(No symbol) [0x00752523]
	(No symbol) [0x0072E27B]
	(No symbol) [0x007C19BF]
	(No symbol) [0x007D4029]
	(No symbol) [0x007BAAC6]
	(No symbol) [0x0078BEFD]
	(No symbol) [0x0078C8FD]
	GetHandleVerifier [0x00B9F143+2981091]
	GetHandleVerifier [0x00BF2FF9+3324825]
	GetHandleVerifier [0x0095B32F+605903]
	GetHandleVerifier [0x00962CBC+637020]
	(No symbol) [0x00866F4D]
	(No symbol) [0x00863DD8]
	(No symbol) [0x00863F75]
	(No symbol) [0x00856406]
	BaseThreadInitThunk [0x77B0FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77DC80CE+286]
	RtlGetAppContainerNamedObjectPath [0x77DC809E+238]


In [1]:
# Simpan Dataframe as JSONL

import csv
import json

# Define the input and output file paths
csv_file_path = '../data/data_context/data_done.csv'  # Replace with your CSV file path
jsonl_file_path = '../data/data_context/data_done.jsonl'  # Replace with your desired output JSONL file path

# Open the CSV file and read it
with open(csv_file_path, mode='r', encoding='utf-8') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    
    # Open the JSONL file for writing
    with open(jsonl_file_path, mode='w', encoding='utf-8') as jsonl_file:
        for row in csv_reader:
            jsonl_file.write(json.dumps(row) + '\n')

print(f"CSV file has been successfully converted to JSON Lines format and saved to {jsonl_file_path}.")


CSV file has been successfully converted to JSON Lines format and saved to ../data/data_context/data_with_context.jsonl.


In [ ]:
company_alias